<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-description" data-toc-modified-id="Project-description-1">Project description</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-2">Data Preparation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Useless-Columns" data-toc-modified-id="Useless-Columns-2.0.0.1">Useless Columns</a></span></li><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-2.0.0.2">Missing Values</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Models-Selection" data-toc-modified-id="Models-Selection-3">Models Selection</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-3.0.1">CatBoost</a></span><ul class="toc-item"><li><span><a href="#Finding-Best-Hyper-Parameters-With-Grid-Search" data-toc-modified-id="Finding-Best-Hyper-Parameters-With-Grid-Search-3.0.1.1">Finding Best Hyper-Parameters With Grid Search</a></span></li><li><span><a href="#CatBoost-With-Best-Hyper-Parameters" data-toc-modified-id="CatBoost-With-Best-Hyper-Parameters-3.0.1.2">CatBoost With Best Hyper-Parameters</a></span></li></ul></li><li><span><a href="#LightGMB:-LGBMRegressor" data-toc-modified-id="LightGMB:-LGBMRegressor-3.0.2">LightGMB: LGBMRegressor</a></span><ul class="toc-item"><li><span><a href="#Finding-Best-Hyper-Parameters-With-Grid-Search" data-toc-modified-id="Finding-Best-Hyper-Parameters-With-Grid-Search-3.0.2.1">Finding Best Hyper-Parameters With Grid Search</a></span></li><li><span><a href="#LGBMRegressor-With-Best-Hyper-Parameters" data-toc-modified-id="LGBMRegressor-With-Best-Hyper-Parameters-3.0.2.2">LGBMRegressor With Best Hyper-Parameters</a></span></li></ul></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-3.0.3">XGBoost</a></span></li></ul></li><li><span><a href="#Dummy-model" data-toc-modified-id="Dummy-model-3.1">Dummy model</a></span></li></ul></li><li><span><a href="#Models-Comparison" data-toc-modified-id="Models-Comparison-4">Models Comparison</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-5">Conclusion</a></span></li></ul></div>

# Project description

Rusty Bargin used car sales service is developing an app to attract new customers. In that app, a user can quickly find out the market value of their car.<br>
<b>The task</b> is to develop a model to determine the value of a car.<br>
<b>Data description</b>
Historic data was provided and it includes: technical specifications, trim versions and prices. The data description:<br>
<i>DateCrawled</i> — date a profile was downloaded from the database<br>
<i>VehicleType</i> — vehicle body type<br>
<i>RegistraionYear</i> — vehicle registration year<br>
<i>Gearbox</i> — gearbox type<br>
<i>Power</i> — power (hp)<br>
<i>Model</i> — vehicle model<br>
<i>Mileage</i> — mileage (measured in km)<br>
<i>RegistrationMonth</i> — vehicle registration month<br>
<i>FuelType</i> — fuel type<br>
<i>Brand</i> — vehicle brand<br>
<i>NotRepaired</i> — vehicle repaired or not<br>
<i>DateCreated</i> — date of the profile creation<br>
<i>NumberOfPictures</i> — number of vehicle pictures<br>
<i>PostalCode</i> — postal code of the profile owner (user)<br>
<i>LastSeen</i> — date of the last activity of the user<br>

# Data Preparation

In [1]:
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import xgboost as xgb

In [2]:
df = pd.read_csv('car_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.sample(2)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
268120,24/03/2016 11:50,400,NaN,2000,manual,0,a6,150000,0,gasoline,audi,yes,24/03/2016 00:00,0,87671,24/03/2016 11:50
43401,08/03/2016 18:39,19777,sedan,2008,auto,326,q7,150000,7,gasoline,audi,no,08/03/2016 00:00,0,45549,15/03/2016 04:46


In [5]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


#### Useless Columns

All columns with dates are not really predictors for this task, so let's just remove them. Also, the column with number of pictures has all zeros. Perhaps this happened due to error while data dump. We will remove it too.

In [6]:
df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures'], axis=1, inplace=True)

#### Missing Values

In [7]:
def count_missing_by_col(df):
    return df.isna().sum().to_frame(name="# of missing values ('NaNs')").T

count_missing_by_col(df)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode
# of missing values ('NaNs'),0,37490,0,19833,0,19705,0,0,32895,0,71154,0


In [8]:
# % of missing data in each column
round(df.isnull().sum() / df.shape[0] * 100, 2).to_frame(name="% of missing values ('NaNs')").T

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode
% of missing values ('NaNs'),0.0,10.58,0.0,5.6,0.0,5.56,0.0,0.0,9.28,0.0,20.08,0.0


There are several methods to deal with missing values. In this project we will fill all missing categorical values with "Nan", e.g. we will create a separate category for them.

In [9]:
categorical = list(df.loc[:, df.dtypes == 'object'].columns.values)
categorical

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [10]:
df[categorical] = df[categorical].fillna('Nan')

# Models Selection

We will split the data into training, validation and test sets in the following ratio: 3:1:1 respectively.

In [11]:
# Splitting into 80% and 20%
remainder, test = train_test_split(df, test_size=0.2, random_state=1)
# splitting 80% into 75% and 25%, thus getting 60% and 20% of total respectively
train, valid = train_test_split(remainder, test_size=0.25, random_state=1)
print(train.shape, test.shape, valid.shape)

(212621, 12) (70874, 12) (70874, 12)


In [12]:
X_train = train.drop(['Price'], axis=1)
X_test = test.drop(['Price'], axis=1)
X_valid = valid.drop(['Price'], axis=1)
y_train = train['Price']
y_test = test['Price']
y_valid = valid['Price']

### CatBoost

We do not have to encode categorical values for the CatBoost model, but we need to indicate what features are categorical. Also, CatBoost will not work with missing data, but we already took care of it.

#### Finding Best Hyper-Parameters With Grid Search

In [13]:
catboost = CatBoostRegressor(cat_features = categorical)

In [14]:
param_grid = {'n_estimators': [100],
        'learning_rate': [0.05, 0.1, 0.15],
        'depth': [6, 8, 10],
        #'random_state': [123],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

In [15]:
grid_search_result = catboost.grid_search(param_grid, X=X_train, y=y_train) #, plot=True) #cv=3)

0:	learn: 6051.4103172	test: 6069.5802599	best: 6069.5802599 (0)	total: 95.6ms	remaining: 9.46s
1:	learn: 5811.1217956	test: 5831.0038673	best: 5831.0038673 (1)	total: 116ms	remaining: 5.68s
2:	learn: 5589.0546955	test: 5609.6450777	best: 5609.6450777 (2)	total: 137ms	remaining: 4.42s
3:	learn: 5378.3155964	test: 5400.9154081	best: 5400.9154081 (3)	total: 151ms	remaining: 3.63s
4:	learn: 5176.8035263	test: 5200.3402741	best: 5200.3402741 (4)	total: 165ms	remaining: 3.14s
5:	learn: 4987.9454866	test: 5012.3913124	best: 5012.3913124 (5)	total: 180ms	remaining: 2.83s
6:	learn: 4809.9297327	test: 4834.7215082	best: 4834.7215082 (6)	total: 195ms	remaining: 2.58s
7:	learn: 4643.0798567	test: 4668.5346486	best: 4668.5346486 (7)	total: 209ms	remaining: 2.41s
8:	learn: 4485.9455689	test: 4512.4448960	best: 4512.4448960 (8)	total: 224ms	remaining: 2.27s
9:	learn: 4339.6109577	test: 4366.9943334	best: 4366.9943334 (9)	total: 239ms	remaining: 2.15s
10:	learn: 4201.4095914	test: 4228.9488421	best: 

91:	learn: 2029.7743759	test: 2063.1060646	best: 2063.1060646 (91)	total: 1.53s	remaining: 133ms
92:	learn: 2027.8577913	test: 2061.0824824	best: 2061.0824824 (92)	total: 1.55s	remaining: 116ms
93:	learn: 2025.8970243	test: 2059.5160444	best: 2059.5160444 (93)	total: 1.56s	remaining: 99.8ms
94:	learn: 2024.5857467	test: 2058.2660897	best: 2058.2660897 (94)	total: 1.58s	remaining: 83.2ms
95:	learn: 2022.7669289	test: 2056.4729228	best: 2056.4729228 (95)	total: 1.6s	remaining: 66.6ms
96:	learn: 2021.0885074	test: 2054.7881255	best: 2054.7881255 (96)	total: 1.61s	remaining: 50ms
97:	learn: 2019.3653647	test: 2053.1440069	best: 2053.1440069 (97)	total: 1.63s	remaining: 33.3ms
98:	learn: 2017.1989037	test: 2051.0844777	best: 2051.0844777 (98)	total: 1.64s	remaining: 16.6ms
99:	learn: 2015.1953958	test: 2049.0240336	best: 2049.0240336 (99)	total: 1.66s	remaining: 0us

bestTest = 2049.024034
bestIteration = 99

0:	loss: 2049.0240336	best: 2049.0240336 (0)	total: 1.75s	remaining: 1m 17s
0:	lea

81:	learn: 1933.6862196	test: 1967.6425763	best: 1967.6425763 (81)	total: 1.34s	remaining: 295ms
82:	learn: 1931.5957922	test: 1965.4532714	best: 1965.4532714 (82)	total: 1.36s	remaining: 280ms
83:	learn: 1929.2503560	test: 1963.3828445	best: 1963.3828445 (83)	total: 1.38s	remaining: 263ms
84:	learn: 1927.2192256	test: 1961.5560233	best: 1961.5560233 (84)	total: 1.4s	remaining: 246ms
85:	learn: 1925.7849950	test: 1960.3160480	best: 1960.3160480 (85)	total: 1.41s	remaining: 229ms
86:	learn: 1924.4295370	test: 1959.1793230	best: 1959.1793230 (86)	total: 1.42s	remaining: 213ms
87:	learn: 1922.6889970	test: 1957.5512205	best: 1957.5512205 (87)	total: 1.44s	remaining: 196ms
88:	learn: 1921.1249172	test: 1956.0719731	best: 1956.0719731 (88)	total: 1.45s	remaining: 179ms
89:	learn: 1919.1488739	test: 1954.0758903	best: 1954.0758903 (89)	total: 1.47s	remaining: 163ms
90:	learn: 1918.2721792	test: 1953.3831220	best: 1953.3831220 (90)	total: 1.48s	remaining: 146ms
91:	learn: 1916.8283240	test: 1

76:	learn: 1893.4079613	test: 1933.8330215	best: 1933.8330215 (76)	total: 1.2s	remaining: 358ms
77:	learn: 1891.1816036	test: 1932.0971912	best: 1932.0971912 (77)	total: 1.22s	remaining: 343ms
78:	learn: 1889.7216276	test: 1930.6688790	best: 1930.6688790 (78)	total: 1.23s	remaining: 327ms
79:	learn: 1887.3994873	test: 1928.1661707	best: 1928.1661707 (79)	total: 1.25s	remaining: 312ms
80:	learn: 1885.9980671	test: 1926.6258321	best: 1926.6258321 (80)	total: 1.26s	remaining: 296ms
81:	learn: 1885.0456283	test: 1925.8957342	best: 1925.8957342 (81)	total: 1.28s	remaining: 280ms
82:	learn: 1883.5601726	test: 1924.9388417	best: 1924.9388417 (82)	total: 1.29s	remaining: 265ms
83:	learn: 1882.5101602	test: 1924.0480767	best: 1924.0480767 (83)	total: 1.31s	remaining: 249ms
84:	learn: 1880.4379770	test: 1922.0258671	best: 1922.0258671 (84)	total: 1.32s	remaining: 233ms
85:	learn: 1878.8420802	test: 1920.4713266	best: 1920.4713266 (85)	total: 1.33s	remaining: 217ms
86:	learn: 1877.2060610	test: 1

70:	learn: 2095.0729215	test: 2126.1747449	best: 2126.1747449 (70)	total: 1.09s	remaining: 446ms
71:	learn: 2089.8340573	test: 2121.0233381	best: 2121.0233381 (71)	total: 1.11s	remaining: 431ms
72:	learn: 2086.0806584	test: 2117.3421012	best: 2117.3421012 (72)	total: 1.12s	remaining: 416ms
73:	learn: 2081.6831240	test: 2112.9642836	best: 2112.9642836 (73)	total: 1.14s	remaining: 400ms
74:	learn: 2077.5249194	test: 2108.4972410	best: 2108.4972410 (74)	total: 1.15s	remaining: 384ms
75:	learn: 2074.4584602	test: 2105.4304642	best: 2105.4304642 (75)	total: 1.17s	remaining: 368ms
76:	learn: 2071.2660751	test: 2102.3660848	best: 2102.3660848 (76)	total: 1.18s	remaining: 353ms
77:	learn: 2068.7206316	test: 2099.8047190	best: 2099.8047190 (77)	total: 1.2s	remaining: 337ms
78:	learn: 2065.6449544	test: 2096.9563131	best: 2096.9563131 (78)	total: 1.21s	remaining: 322ms
79:	learn: 2062.4253762	test: 2093.8678658	best: 2093.8678658 (79)	total: 1.22s	remaining: 306ms
80:	learn: 2059.4951813	test: 2

...

46:	learn: 1827.4167460	test: 1893.5286085	best: 1893.5286085 (46)	total: 1.65s	remaining: 1.86s
47:	learn: 1823.5781338	test: 1891.0398445	best: 1891.0398445 (47)	total: 1.68s	remaining: 1.82s
48:	learn: 1820.2148053	test: 1888.8402063	best: 1888.8402063 (48)	total: 1.72s	remaining: 1.79s
49:	learn: 1816.9450738	test: 1886.6111706	best: 1886.6111706 (49)	total: 1.75s	remaining: 1.75s
50:	learn: 1815.3857536	test: 1885.7997525	best: 1885.7997525 (50)	total: 1.78s	remaining: 1.71s
51:	learn: 1812.8239237	test: 1884.1758643	best: 1884.1758643 (51)	total: 1.81s	remaining: 1.67s
52:	learn: 1810.1292771	test: 1881.9592486	best: 1881.9592486 (52)	total: 1.84s	remaining: 1.63s
53:	learn: 1808.3218370	test: 1880.5274171	best: 1880.5274171 (53)	total: 1.87s	remaining: 1.59s
54:	learn: 1804.4176115	test: 1878.1603234	best: 1878.1603234 (54)	total: 1.9s	remaining: 1.56s
55:	learn: 1801.4187186	test: 1875.5388888	best: 1875.5388888 (55)	total: 1.93s	remaining: 1.52s
56:	learn: 1798.7290680	test: 1

34:	learn: 2413.3343025	test: 2447.3265537	best: 2447.3265537 (34)	total: 1.33s	remaining: 2.47s
35:	learn: 2382.4399001	test: 2416.5185504	best: 2416.5185504 (35)	total: 1.38s	remaining: 2.45s
36:	learn: 2355.4857566	test: 2389.7806663	best: 2389.7806663 (36)	total: 1.42s	remaining: 2.41s
37:	learn: 2329.5424089	test: 2364.5612233	best: 2364.5612233 (37)	total: 1.46s	remaining: 2.37s
38:	learn: 2305.2731839	test: 2340.4240648	best: 2340.4240648 (38)	total: 1.49s	remaining: 2.33s
39:	learn: 2283.2995634	test: 2318.5633442	best: 2318.5633442 (39)	total: 1.53s	remaining: 2.29s
40:	learn: 2262.7436551	test: 2298.4616617	best: 2298.4616617 (40)	total: 1.57s	remaining: 2.27s
41:	learn: 2243.1127103	test: 2279.2779157	best: 2279.2779157 (41)	total: 1.61s	remaining: 2.23s
42:	learn: 2225.2219403	test: 2261.6669109	best: 2261.6669109 (42)	total: 1.65s	remaining: 2.19s
43:	learn: 2208.6158673	test: 2245.0528450	best: 2245.0528450 (43)	total: 1.69s	remaining: 2.15s
44:	learn: 2189.7077363	test: 

23:	learn: 2133.1220825	test: 2171.4952323	best: 2171.4952323 (23)	total: 767ms	remaining: 2.43s
24:	learn: 2108.3212537	test: 2147.3449376	best: 2147.3449376 (24)	total: 796ms	remaining: 2.39s
25:	learn: 2087.3312393	test: 2126.1786150	best: 2126.1786150 (25)	total: 823ms	remaining: 2.34s
26:	learn: 2068.7143694	test: 2107.6976663	best: 2107.6976663 (26)	total: 848ms	remaining: 2.29s
27:	learn: 2052.9561547	test: 2092.6677882	best: 2092.6677882 (27)	total: 875ms	remaining: 2.25s
28:	learn: 2037.4452526	test: 2079.2137950	best: 2079.2137950 (28)	total: 900ms	remaining: 2.2s
29:	learn: 2023.6367047	test: 2067.0145309	best: 2067.0145309 (29)	total: 925ms	remaining: 2.16s
30:	learn: 2007.6384462	test: 2051.2008387	best: 2051.2008387 (30)	total: 952ms	remaining: 2.12s
31:	learn: 1993.0990831	test: 2037.4235059	best: 2037.4235059 (31)	total: 979ms	remaining: 2.08s
32:	learn: 1981.3359420	test: 2026.1100288	best: 2026.1100288 (32)	total: 1.01s	remaining: 2.05s
33:	learn: 1972.1458192	test: 2

10:	learn: 2428.9727401	test: 2461.6450096	best: 2461.6450096 (10)	total: 333ms	remaining: 2.69s
11:	learn: 2341.3819669	test: 2374.9046467	best: 2374.9046467 (11)	total: 364ms	remaining: 2.67s
12:	learn: 2272.6583154	test: 2307.7292719	best: 2307.7292719 (12)	total: 391ms	remaining: 2.61s
13:	learn: 2217.1927251	test: 2253.8194494	best: 2253.8194494 (13)	total: 418ms	remaining: 2.57s
14:	learn: 2164.4428256	test: 2202.0452654	best: 2202.0452654 (14)	total: 444ms	remaining: 2.52s
15:	learn: 2121.0191560	test: 2159.0021486	best: 2159.0021486 (15)	total: 473ms	remaining: 2.48s
16:	learn: 2087.2613774	test: 2126.4864579	best: 2126.4864579 (16)	total: 502ms	remaining: 2.45s
17:	learn: 2059.5741031	test: 2099.5140737	best: 2099.5140737 (17)	total: 527ms	remaining: 2.4s
18:	learn: 2037.6441801	test: 2079.5860213	best: 2079.5860213 (18)	total: 559ms	remaining: 2.38s
19:	learn: 2012.5895319	test: 2054.4142497	best: 2054.4142497 (19)	total: 589ms	remaining: 2.35s
20:	learn: 1992.8366820	test: 2

0:	learn: 5531.0012289	test: 5532.0055525	best: 5532.0055525 (0)
1:	learn: 4887.7540225	test: 4886.0615669	best: 4886.0615669 (1)
2:	learn: 4352.3080415	test: 4350.3362267	best: 4350.3362267 (2)
3:	learn: 3913.6748860	test: 3912.2907757	best: 3912.2907757 (3)
4:	learn: 3554.1806631	test: 3552.7933796	best: 3552.7933796 (4)
5:	learn: 3259.7684098	test: 3259.3797426	best: 3259.3797426 (5)
6:	learn: 3016.7614380	test: 3016.9239333	best: 3016.9239333 (6)
7:	learn: 2824.3491192	test: 2825.6426220	best: 2825.6426220 (7)
8:	learn: 2670.7295181	test: 2673.3220965	best: 2673.3220965 (8)
9:	learn: 2540.2376206	test: 2544.3666171	best: 2544.3666171 (9)
10:	learn: 2426.3691151	test: 2431.9781108	best: 2431.9781108 (10)
11:	learn: 2336.7347510	test: 2343.1030373	best: 2343.1030373 (11)
12:	learn: 2264.2742189	test: 2272.1563562	best: 2272.1563562 (12)
13:	learn: 2209.2786842	test: 2217.9907889	best: 2217.9907889 (13)
14:	learn: 2161.7675051	test: 2171.7046188	best: 2171.7046188 (14)
15:	learn: 2118

In [16]:
grid_search_result['params']

{'depth': 10, 'l2_leaf_reg': 1, 'iterations': 100, 'learning_rate': 0.15}

#### CatBoost With Best Hyper-Parameters

In [17]:
catboost_best = CatBoostRegressor(iterations=100, 
                                  learning_rate=0.15, 
                                  depth=10,
                                  l2_leaf_reg= 1,
                                  random_seed=123, 
                                  cat_features=categorical)

In [18]:
%%time
catboost_best.fit(X_train, y_train, cat_features=categorical, verbose=100)

0:	learn: 4051.3871512	total: 34.6ms	remaining: 3.42s
99:	learn: 1684.1539358	total: 3.34s	remaining: 0us
CPU times: user 20.1 s, sys: 795 ms, total: 20.9 s
Wall time: 3.75 s


In [19]:
%%time
catboost_predictions_valid = catboost_best.predict(X_valid)

CPU times: user 163 ms, sys: 5.36 ms, total: 168 ms
Wall time: 133 ms


In [20]:
rmse_cat_valid = mean_squared_error(y_valid, catboost_predictions_valid)**0.5
print("RMSE on validation set: %.2f" % rmse_cat_valid)

RMSE on validation set: 1811.47


In [21]:
rmse_cat_test = mean_squared_error(y_test, catboost_best.predict(X_test))**0.5
print("RMSE on validation set: %.2f" % rmse_cat_test)

RMSE on validation set: 1790.44


### LightGMB: LGBMRegressor

Another model that we will try is LGBMRegressor. Similar to CatBoost this model can handle categorical values without having to encode them, but we do need to change the type of categorical values from "object" to "category". Let's prepare features for LGBMRegressor. 

In [22]:
categorical

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [23]:
for feature in categorical:
    X_train[feature] = pd.Series(X_train[feature], dtype="category")
    X_valid[feature] = pd.Series(X_valid[feature], dtype="category")
    X_test[feature] = pd.Series(X_test[feature], dtype="category")

#### Finding Best Hyper-Parameters With Grid Search

In [24]:
lgbm_model = LGBMRegressor()

In [25]:
param_grid = {
    'learning_rate': [0.05, 0.1, 0.15],
    'n_estimators': [100, 200, 300, 400], #Number of boosted trees to fit.
    'max_depth': [5, 10, 15, 20, 25] #Maximum tree depth for base learners, <=0 means no limit.
}

In [26]:
grid_search_lgbm = GridSearchCV(estimator=lgbm_model,
                                param_grid=param_grid,
                                cv=4,
                                scoring='neg_root_mean_squared_error')

In [27]:
grid_search_lgbm.fit(X_train, y_train, categorical_feature=categorical)

GridSearchCV(cv=4, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.05, 0.1, 0.15],
                         'max_depth': [5, 10, 15, 20, 25],
                         'n_estimators': [100, 200, 300, 400]},
             scoring='neg_root_mean_squared_error')

In [28]:
grid_search_lgbm.best_params_

{'learning_rate': 0.15, 'max_depth': 15, 'n_estimators': 400}

#### LGBMRegressor With Best Hyper-Parameters

In [29]:
lgbm_best = LGBMRegressor(learning_rate=0.15, max_depth=15, n_estimators=400)

In [30]:
%%time
lgbm_best.fit(X_train, y_train, categorical_feature=categorical) 

CPU times: user 7.87 s, sys: 126 ms, total: 8 s
Wall time: 1.06 s


LGBMRegressor(learning_rate=0.15, max_depth=15, n_estimators=400)

In [31]:
%%time
lgbm_predictions_valid = lgbm_best.predict(X_valid)

CPU times: user 2.05 s, sys: 21.8 ms, total: 2.07 s
Wall time: 297 ms


In [32]:
rmse_lgbm_valid = mean_squared_error(y_valid, lgbm_predictions_valid)**0.5
print("RMSE on validation set: %.2f" % rmse_lgbm_valid)

RMSE on validation set: 1739.70


In [33]:
rmse_lgbm_test = mean_squared_error(y_test, lgbm_best.predict(X_test))**0.5
print("RMSE on validation set: %.2f" % rmse_lgbm_test)

RMSE on validation set: 1716.87


### XGBoost

XGBBoost requires categorical values to be encoded. 

In [34]:
categorical

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [35]:
label_encoder = LabelEncoder()

for col in categorical:
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.fit_transform(X_test[col])
    X_valid[col] = label_encoder.fit_transform(X_valid[col])

In [36]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dvalid= xgb.DMatrix(data=X_valid, label=y_valid)
dtest=xgb.DMatrix(data=X_test, label=y_test)

In [37]:
params = {'objective':'reg:linear', 'colsample_bytree': 0.3, 'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10, 'n_estimators': 10, 'random_state':123}

In [38]:
%%time
bst = xgb.train(params, dtrain)

[14:47:28] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[14:47:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


CPU times: user 2.05 s, sys: 53.1 ms, total: 2.11 s
Wall time: 287 ms


In [39]:
%%time
xgb_predictions_valid = bst.predict(dvalid)

CPU times: user 27.9 ms, sys: 2.51 ms, total: 30.4 ms
Wall time: 4.74 ms


In [40]:
rmse_xgb_valid = mean_squared_error(y_valid, xgb_predictions_valid)**0.5
print("RMSE validation set: %.2f" % rmse_xgb_valid)

RMSE validation set: 3834.46


In [41]:
rmse_xgb_test = round(mean_squared_error(y_test, bst.predict(dtest))**0.5, 0)
print("RMSE test set: %.2f" % rmse_xgb_test)

RMSE test set: 3833.00


## Dummy model

In [42]:
mean = y_train.mean()
length = X_test.shape[0]
dummy_predictions = np.full((length,1), mean)
dummy_rmse = mean_squared_error(dummy_predictions, y_test)**0.5
print('RMSE dummy model:',dummy_rmse)

RMSE dummy model: 4519.468512614231


# Models Comparison

Among the three models that we have tried the best accuracy was achieved with the Random Forest Classifier model with the number of trees in the forest set to 25.

In [45]:
d = {'RSME_valid': [rmse_cat_valid, rmse_lgbm_valid, rmse_xgb_valid, dummy_rmse],
     'RSME_test': [rmse_cat_test, rmse_lgbm_test, rmse_xgb_test, dummy_rmse],
     'Training time': ['3-4 s', '1-1.1 s', '290-330 ms', '--'],
     'Prediction time': ['110-140 ms', '290-300 ms', '5-9 ms', '--']} 
     
final = pd.DataFrame(data=d, index = ['CatBoostRegressor', 'LGBMRegressor', 'XGb',  'Dummy Model'])
display(final)

,RSME_valid,RSME_test,Training time,Prediction time
CatBoostRegressor,1811.468574,1790.444140,3-4 s,110-140 ms
LGBMRegressor,1739.699177,1716.868107,1-1.1 s,290-300 ms
XGb,3834.456752,3833.000000,290-330 ms,5-9 ms
Dummy Model,4519.468513,4519.468513,--,--


Among all three models LGBMRegressor demonstrates best accuracy and second least run time for training and prediction. XGB model is seen to be least accurate: it does a bit better than a dummy model, which just predicts the mean value. Plus XGB model's run time is the highest among all.

# Conclusion

 1. We looked into the given data and found some missing values. All missing values were categorical. We chose to deal with missing categorical values by creating a new category 'Nan'.<br>
 2. We compared three models and chose the best model - LGBMRegressor, whose RMSE metric is the lowest, meaning the best. Also, it demonstrates good run time in terms of speed of prediction.<br>
The CatBoostRegressor is the runner up. It also demonstrated very good RMSE metric and acceptable time.